In [1]:
import pandas as pd
import pickle
import mlflow

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer
import uuid
import os



In [2]:
year = 2021
month = 3
taxi_type='green'
input_file = f'https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}/tripdata_{year:04d}-{month:02d}.parquet'
RUN_ID = os.getenv('RUN_ID','8cf38dfad1a24ceb9b58de03cc679dd9')
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('random-forest-experiment')

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='random-forest-experiment', tags={}>

In [3]:
def generate_uuids(n):
    #n = len(df)
    ride_ids = []

    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))  

    return ride_ids



def read_dataframe(path):
    df = pd.read_parquet(path)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    
    df.duration = df.duration.dt.total_seconds() / 60
    
    df = df[(df.duration > 1) & (df.duration < 60)]
    
    categorical = ['PULocationID','DOLocationID']
    
    df[categorical] = df[categorical].astype(str)
    
    return df

def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    #df = df[categorical + numerical]
    return dicts


In [6]:
def load_model(run_id):
    logged_model = f'runs:/{run_id}/models'
    model = mlflow.pyfunc.load_model(logged_model)
    return model

def apply_model(input_file,run_id,output_file):
    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)
    model = load_model(run_id)
    with open('DictVectorizer.bin','rb') as f:
        dv = pickle.load(f)
        x = dv.transform(dicts)
        #logged_model = 'runs:/8cf38dfad1a24ceb9b58de03cc679dd9/models'

        y_pred = model.predict(x)

        df_result = pd.DataFrame()

        df_result['ride_id'] = generate_uuids(len(df))
        df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
        df_result['PULocationID'] = df['PULocationID']
        df_result['DOLocationID'] = df['DOLocationID']
        df_result['actual_duration'] = df['duration']
        df_result['predicted_duration'] = y_pred
        df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
        df_result['model_version'] = run_id

        df.to_parquet(output_file,index=False)





In [7]:
apply_model(input_file,RUN_ID,output_file)